In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, TimeDistributed, Flatten, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.utils import to_categorical

# تعريف نموذج CNN-LSTM
def create_cnn_lstm_model(input_shape, num_classes):
    model = Sequential()

    # طبقات CNN
    model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'), input_shape=input_shape))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu', padding='same')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Flatten()))

    # طبقات LSTM
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(64))
    model.add(Dropout(0.5))

    # طبقات Fully Connected
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    # تجميع النموذج
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# تحضير البيانات (هذا مثال، يجب استبداله ببياناتك الخاصة)
def load_example_data():
    # مثال: إنشاء بيانات وهمية
    num_samples = 200  # عدد العينات
    num_frames = 30    # عدد الإطارات في كل فيديو
    height, width, channels = 64, 64, 3  # أبعاد الإطارات
    num_classes = 10   # عدد الفئات (العلامات)

    X_train = np.random.rand(num_samples, num_frames, height, width, channels)  # بيانات تدريب
    y_train = np.random.randint(0, num_classes, num_samples)  # تسميات تدريب

    X_test = np.random.rand(num_samples // 2, num_frames, height, width, channels)  # بيانات اختبار
    y_test = np.random.randint(0, num_classes, num_samples // 2)  # تسميات اختبار

    # تحويل التسميات إلى one-hot encoding
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

    return X_train, X_test, y_train, y_test

# تحميل البيانات
X_train, X_test, y_train, y_test = load_example_data()

# تعريف شكل المدخلات وعدد الفئات
input_shape = (None, 64, 64, 3)  # (عدد الإطارات, ارتفاع, عرض, قنوات)
num_classes = 10

# إنشاء النموذج
model = create_cnn_lstm_model(input_shape, num_classes)

# تدريب النموذج
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# تقييم النموذج
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# حفظ النموذج
model.save('sign_language_cnn_lstm_v2.h5')

Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 293s 37s/step - accuracy: 0.1220 - loss: 2.3503 - val_accuracy: 0.0800 - val_loss: 2.4199
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 273s 39s/step - accuracy: 0.1575 - loss: 2.2954 - val_accuracy: 0.1400 - val_loss: 2.3018
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 262s 37s/step - accuracy: 0.0957 - loss: 2.3176 - val_accuracy: 0.0800 - val_loss: 2.2826
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 252s 36s/step - accuracy: 0.1017 - loss: 2.3338 - val_accuracy: 0.0800 - val_loss: 2.2836
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 272s 39s/step - accuracy: 0.1112 - loss: 2.3695 - val_accuracy: 0.1500 - val_loss: 2.2849
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 250s 35s/step - accuracy: 0.0856 - loss: 2.3475 - val_accuracy: 0.1500 - val_loss: 2.2951
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 272s 39s/step - accuracy: 0.1176 - loss: 2.2951 - val_accuracy: 0.1500 - val_loss: 2.2947
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 257s 36s/step - accuracy: 0.1322 - loss: 2.3011 - val_accuracy: 0.1500 - val_loss: